In [42]:
%run 'init.ipynb'
%run 'functions.ipynb'
%run 'geometry_checks.ipynb'

225.53


In [43]:
N = 1000000
mass_fv = 40000

materials = [
    "SS_InnerCryostat",
    "SS_OuterCryostat",
    "OuterCryostatReflector",
    "SS_BellPlate",
    "SS_BellSideWall",
    "PmtTpc",
    "Copper_TopRing",
    "Copper_LowerRing",
    "Teflon_Pillar_",
    "Copper_FieldGuard_",
    "Copper_FieldShaperRing_",
    "Electrodes",
    "Teflon_BottomTPC",
    "Teflon_TPC",   
    "Top_PTFE_Frame",
    "Copper_BottomPmtPlate",
    "Copper_TopPmtPlate",
    ]



DATE = 'XENONnT_2019-11-07'

isotope_array = ["U238Pb206",
                "K40",
                "Th228Pb208",
                "U235Pb207",
                "Co60",
                "Cs137",]
num_file = '1'

array = np.arange(1, 2)



In [12]:
def calculate_events(data):
    ss = single_scatter(data)
    fv_ = fv(ss)
    en = energy(fv_, 1, 12)
    events_passing = len(en)
    return events_passing


In [24]:
material ="OuterCryostat_Elongation"
def calculate_rate(data, material,isotope,  n_gen, m_fv):
    n_events = calculate_events(data)
    E = 12
    p = get_parameters(material)
    mass = p['mass']

    contamination = p[isotope]
    err = "err_"+isotope
    err = str(err)
    if (contamination == 0):
        print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|","No contamination value", "|" )
  
    else:
        time = n_gen / (mass * contamination)
        rate = n_events/(time * m_fv * E)
        alpha = mass / (m_fv * E * n_gen)
        err = "err_"+isotope
        err = str(err)
        error_rate = alpha * np.sqrt((n_events * p[err])**2 + (contamination*np.sqrt(n_events))**2)
        print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|", rate, "+-", error_rate, "|")


In [14]:
material ="OuterCryostat_Elongation"
def calculate_rate_TeflonTPC(data, material, n_gen, m_fv):
    print("MODIFY ME!!!!!")
    n_events = calculate_events(data)
    E = 12
    n1 = n_events * 0.7571
    n2 = n_events * 0.2429
    material = "Teflon_TPC_Sliding"
    p = get_parameters(material)
    mass = p['mass']
    print("^material ^isotope  ^ contamination ^ rate ^ ")
    
    for isotope in isotope_array:
        contamination = p[isotope]
        if (contamination == 0):
            print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|","No contamination value", "|" )
  
        else:
            time = n_gen / (mass * contamination)
            rate = n1/(time * m_fv * E)
            alpha = mass / (m_fv * E * n_gen)
            err = "err_"+isotope
            err = str(err)
            error_rate = alpha * np.sqrt((n1 * p[err])**2 + (contamination*np.sqrt(n1))**2)
            print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|", rate, "+-", error_rate, "|")
            
    material = "Teflon_TPC_Fixed"
    p = get_parameters(material)
    mass = p['mass']
    print("^material ^isotope  ^ contamination ^ rate ^ ")
    
    for isotope in isotope_array:
        contamination = p[isotope]
        if (contamination == 0):
            print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|","No contamination value", "|" )
  
        else:
            time = n_gen / (mass * contamination)
            rate = n2/(time * m_fv * E)
            alpha = mass / (m_fv * E * n_gen)
            err = "err_"+isotope
            err = str(err)
            error_rate = alpha * np.sqrt((n2 * p[err])**2 + (contamination*np.sqrt(n2))**2)
            print("|", material, "|", isotope, "|", p[isotope],"+-", p[err],"|", rate, "+-", error_rate, "|")

In [29]:
DATE = 'XENONnT_2019-11-04'
materials = ["SS_OuterCryostat",
    #"Top_PTFE_Frame"
    ]

for material in materials:
    print("^material ^isotope  ^ contamination ^ rate ^ ")
    for isotope in isotope_array:

        if material == "Top_PTFE_Frame":
            data = get_TopPFTE()
            material = "Top_PTFE_Frame"
            print("^material ^isotope  ^ contamination ^ rate ^ ")
            calculate_rate(data, material, N, 4000)

        elif material == "Electrodes":
            data = get_electrodes()
            material = "SS_Electrodes"
            calculate_rate(data, material, N, 4000)

        elif material == 'SS_OuterCryostat':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            flange1, flange2, flange3, shell, elongation = divide_outercryo(data)
            flange = pd.DataFrame()
            flanges = pd.concat([flange1, flange2, flange3])
            calculate_rate(flanges, "OuterCryostat_flanges",isotope, N, mass_fv)
            calculate_rate(shell, "OuterCryostatShell",isotope, N, mass_fv)
            cd /sccalculate_rate(elongation, "OuterCryostat_Elongation",isotope, N, mass_fv)

        elif material == 'SS_InnerCryostat':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            flange1_in, flange2_in, flange3_in, shell_in, elongation_in = divide_innercryo(data)
            flange_in = pd.concat([flange1_in, flange2_in, flange3_in])
            calculate_rate(flange_in, "SS_InnerCryostatFlanges", N, mass_fv)
            calculate_rate(shell_in, "SS_InnerCryostatShell", N, mass_fv)
            calculate_rate(elongation_in, "SS_InnerCryostat_bottomDome", N, mass_fv)

        elif material == 'SS_BellSideWall':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            bottomLip, bellWall = divide_bellWall(data)
            calculate_rate(bottomLip, "SS_BellSideWallBottomLip", N, mass_fv)
            calculate_rate(bellWall, "SS_BellSideWall", N, mass_fv)

        elif material == 'Teflon_TPC':
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            calculate_rate_TeflonTPC(data, material, N, mass_fv)

        else: 
            rootfile = '/userdata/arocchetti/' + DATE + '/'+ material + '/' + isotope + '/' + 'output_'+ material + '_' + isotope +  '_' + num_file+'_Sort' + '.root'
            data = get_xyz(rootfile)
            calculate_rate(data, material, N, mass_fv)



^material ^isotope  ^ contamination ^ rate ^ 
| OuterCryostat_flanges | U238Pb206 | 1.4 +- 0.4 | 0.0 +- 0.0 |
| OuterCryostat_flanges | K40 | 5.6 +- 0 | 0.0 +- 0.0 |
| OuterCryostat_flanges | Th228Pb208 | 4.5 +- 0.6 | 0.0 +- 0.0 |
| OuterCryostat_flanges | U235Pb207 | 0.06 +- 0.02 | 0.0 +- 0.0 |
| OuterCryostat_flanges | Co60 | 37.3 +- 0.9 | 0.0 +- 0.0 |
| OuterCryostat_flanges | Cs137 | 1.5 +- 0 | 0.0 +- 0.0 |
